In [1]:
# 导入mxnet
import mxnet as mx

# 设置随机种子
mx.random.seed(2)

from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import image
from utils import load_data_fashion_mnist, accuracy, evaluate_accuracy

## 获取和读取数据

In [2]:
# 批数据大小
batch_size = 256

# 获取训练数据和测试数据
train_data, test_data = load_data_fashion_mnist(batch_size)

## 定义模型

In [3]:
# 按顺序堆叠网络层
net = gluon.nn.Sequential()

# name_scope作用, 方便管理参数名称
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(256, activation='relu'))
    net.add(gluon.nn.Dense(10))

# 参数初始化
net.initialize()

## Softmax和交叉熵损失函数

In [4]:
# 定义交叉熵损失
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## 优化

In [5]:
# 定义训练器和优化方法
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练

In [6]:
# 定义迭代周期
epochs = 5

# 训练
for epoch in range(epochs):
    # 训练损失
    train_loss = 0.0
    # 训练集准确率
    train_acc = 0.0
    # 迭代训练
    for data, label in train_data:
        # 记录梯度
        with autograd.record():
            # 计算输出
            output = net(data)
            # 计算损失
            loss = softmax_cross_entropy(output, label)
        # 反向传播求梯度
        loss.backward()
        # 梯度下降
        trainer.step(batch_size)
        # 总的训练损失
        train_loss += nd.mean(loss).asscalar()
        # 总的训练准确率
        train_acc += accuracy(output, label)
    
    # 测试集的准确率
    test_acc = evaluate_accuracy(test_data, net)
    
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

Epoch 0. Loss: 0.819048, Train acc 0.728666, Test acc 0.768530
Epoch 1. Loss: 0.550646, Train acc 0.808644, Test acc 0.823618
Epoch 2. Loss: 0.488554, Train acc 0.829210, Test acc 0.845553
Epoch 3. Loss: 0.457407, Train acc 0.839493, Test acc 0.842448
Epoch 4. Loss: 0.438059, Train acc 0.845486, Test acc 0.852063
